# Code Generator

The requirement: use an Open Source model to generate high performance C++ code from Python code

To replicate this, you'll need to set up a HuggingFace endpoint as I do in the video. It's simple to do, and it's quite satisfying to see the results!

It's also an important part of your learning; this is the first example of deploying an open source model to be behind an API. We'll return to this in Week 8, but this should plant a seed in your mind for what's involved in moving open source models into production.

Added the use of inference providers that was introduced recently by Hugging Face to convert the code.
Improved the user prompt to include algorithic efficeiny and performance optimization.

Added Java as a conversion option.

Note: C++ commands work on windows environment.


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important - Pause Endpoints when not in use</h1>
            <span style="color:#900;">
            If you do decide to use HuggingFace endpoints for this project, you should stop or pause the endpoints when you are done to avoid accruing unnecessary running cost. The costs are very low as long as you only run the endpoint when you're using it. Navigate to the HuggingFace endpoint UI <a href="https://ui.endpoints.huggingface.co/">here,</a> open your endpoint, and click Pause to put it on pause so you no longer pay for it.  
Many thanks to student John L. for raising this.
<br/><br/>
In week 8 we will use Modal instead of HuggingFace endpoints; with Modal you only pay for the time that you use it and you should get free credits.
            </span>
        </td>
    </tr>
</table>

In [0]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess, re

In [0]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [0]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [0]:
def system_prompt_for(language: str) -> str:
    system_prompt = (
        f"You are an assistant that reimplements Python code in high performance {language.upper()} for an Windows intel i7."
        f"Respond only with {language.upper()} code; use comments sparingly and do not provide any explanation other than occasional comments."
        f"The {language.upper()} response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly."
    )
    return system_prompt

In [0]:
#updated the original prompt to include algorithic efficeiny and performance optimization
def user_prompt_for(python: str, language: str) -> str:
    if language.lower() not in {"cpp", "java"}:
        raise ValueError("Unsupported language. Please choose 'C++' or 'Java'.")
    
    optimization_notes = {
        "cpp": (
            "- Use `int64_t` instead of `int` where necessary to prevent overflows.\n"
            "- Ensure random number generation in C++ matches Python's output as closely as possible.\n"
            "- Avoid undefined behavior, such as bit shifts that exceed type width (`1UL << 32` is incorrect for `uint32_t`).\n"
            "- Utilize `std::vector` for dynamic arrays and prefer preallocation for efficiency.\n"
            "- Consider `std::array` or `std::span` when fixed-size arrays are sufficient.\n"
            "- Optimize with **SIMD**, cache-friendly structures, and memory alignment where necessary.\n"
        ),
        "java": (
            "- Use `long` instead of `int` where necessary to prevent overflows.\n"
            "- Ensure random number generation in Java matches Python's output as closely as possible.\n"
            "- Use `ArrayList` instead of primitive arrays if dynamic resizing is needed.\n"
            "- Utilize `BigInteger` if handling large numbers that could exceed `long`.\n"
            "- Optimize with **parallel streams** (`IntStream.parallel()`) and **efficient data structures** (`HashMap`, `LinkedList`, etc.).\n"
        )
    }

    user_prompt = (
        f"First, analyze the given Python code to understand its core purpose and algorithmic approach. "
        f"Then, implement a {language} solution that achieves the same output while prioritizing:\n"
        "1. **Algorithmic Efficiency** - Optimize time and space complexity, even if it means using a different approach.\n"
        "2. **Numerical Correctness** - Prevent integer overflows, use appropriate data types (`long`, `BigInteger`, etc.), "
        "and ensure correct handling of edge cases.\n"
        "3. **Performance Optimization** - Utilize language-specific features for efficiency.\n\n"
        
        "### **Important Notes:**\n"
        + optimization_notes[language.lower()] +
        "\n### **Expected Response:**\n"
        f"Respond **only with {language} code**, including all necessary imports and ensuring the output matches the Python version exactly.\n\n"
        
        "Here's the Python code to analyze and optimize:\n\n"
        + python
    )
    
    return user_prompt


In [0]:
def messages_for(python, language="cpp"):
    return [
        {"role": "system", "content": system_prompt_for(language)},
        {"role": "user", "content": user_prompt_for(python, language)}
    ]

In [0]:
# write to a file called optimized.cpp

def write_output(code, file_name):
    with open(file_name, "w") as f:
        f.write(code)

In [0]:
def optimize_gpt(python, language="cpp"):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python, language), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    file_name= f"optimized.{language}"
    if language == "java":
        # Extract class name from Java code
        match = re.search(r"\b(public\s+)?class\s+(\w+)", reply)
        class_name = match.group(2) if match else "OptimizedJava"
        file_name = f"{class_name}.java"
    else:
        file_name = f"optimized.{language}"
    write_output(reply, file_name)

In [0]:
def optimize_claude(python, language="cpp"):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python, language)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    if language == "java":
        # Extract class name from Java code
        match = re.search(r"\b(public\s+)?class\s+(\w+)", reply)
        class_name = match.group(2) if match else "OptimizedJava"
        file_name = f"{class_name}.java"
    else:
        file_name = f"optimized.{language}"
    write_output(reply, file_name)

In [0]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [0]:
exec(pi)

In [0]:
optimize_gpt(pi, "java")

In [0]:
exec(pi)

In [0]:
!g++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!optimized.exe

In [0]:
optimize_claude(pi)

In [0]:
!g++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!optimized.exe

In [0]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [0]:
exec(python_hard)

In [0]:
optimize_gpt(python_hard)

In [0]:
!g++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!optimized.exe

In [0]:
optimize_claude(python_hard)

In [0]:
!g++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!optimized.exe

In [0]:
def stream_gpt(python, language="cpp"):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python, language), stream=True)
    reply = ""
    code_block = f"```{language}\n"
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        cleaned_reply = reply.replace(code_block,'').replace('```','')
        yield cleaned_reply

In [0]:
def stream_claude(python, language="cpp"):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python, language)}],
    )
    reply = ""
    code_block = f"```{language}\n"
    with result as stream:
        for text in stream.text_stream:
            reply += text
            cleaned_reply = reply.replace(code_block,'').replace('```','')
            yield cleaned_reply

In [0]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [0]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

In [0]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [0]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
.java {background-color: #306775;}
"""

In [0]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

In [0]:
from huggingface_hub import login, InferenceClient

In [0]:
hf_token = os.environ['HF_TOKEN']
login(hf_token)

In [0]:
code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
code_gemma = "google/codegemma-7b-it"
messages=messages_for(pi)

In [0]:
client = InferenceClient(
	provider="sambanova",
	api_key=hf_token
)
stream = client.chat.completions.create(
	model="Qwen/Qwen2.5-Coder-32B-Instruct", 
	messages=messages, 
	max_tokens=500,
	stream=True
)

for chunk in stream:
    print(chunk.choices[0].delta.content, end="")

In [0]:
# using inference providers
def stream_code_qwen(python):
    messages = messages_for(python)
    client = InferenceClient(
    	provider="sambanova",
    	api_key=hf_token
    )
    stream = client.chat.completions.create(
    	model="Qwen/Qwen2.5-Coder-32B-Instruct", 
    	messages=messages, 
    	max_tokens=500,
    	stream=True
    )
    result = ""
    for chunk in stream:
        if chunk.choices and chunk.choices[0].delta.content:
            result += chunk.choices[0].delta.content
            yield result

In [0]:
def optimize(python, model, language):
    if model=="GPT":
        result = stream_gpt(python, language)
    elif model=="Claude":
        result = stream_claude(python, language)
    elif model=="CodeQwen":
        result = stream_code_qwen(python, language)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far    

In [0]:
def select_sample_program(sample_program):
    if sample_program=="pi":
        return pi
    elif sample_program=="python_hard":
        return python_hard
    else:
        return "Type your Python program here"

In [0]:
def execute_cpp(code):
        write_output(code, "optimized.exe")
        try:
            compile_cmd = ["g++", "-Ofast", "-std=c++17", "-march=native", "-mtune=intel", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["optimized.exe"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [0]:
def execute_java(code):
    # Extract the class name from the Java code
    match = re.search(r"\b(public\s+)?class\s+(\w+)", code)
    class_name = match.group(2) if match else "OptimizedJava"

    file_name = f"{class_name}.java"
    write_output(code, file_name)
    try:
        compile_cmd =["javac", file_name]
        subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
        run_cmd = ["java", class_name]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error during compilation or execution:\n{e.stderr}"

In [0]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++ or Java")
    #input and output
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        converted_code = gr.Textbox(label="Converted code:", lines=10)
        # java = gr.Textbox(label="Java code:", lines=10)
    #sample programs
    with gr.Row():
        with gr.Column():
            sample_program = gr.Radio(["pi", "python_hard"], label="Sample program", value="python_hard")
    #select model and language
    with gr.Row():
        with gr.Column():
            model = gr.Dropdown(["GPT", "Claude", "CodeQwen"], label="Select model", value="GPT")
            language = gr.Dropdown(["C++","Java"], label="Select language", value="C++")
    with gr.Row():
        convert = gr.Button("Convert")
    #Code execution
    with gr.Row():
        python_run = gr.Button("Run Python")
        converted_run = gr.Button("Run converted code")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        output = gr.TextArea(label="Converted code result:", elem_classes=["cpp"])
        
    # Function to convert Python code based on language
    def convert_code(python_code, model, selected_language):
        if selected_language == "C++":
            for chunk in optimize(python_code, model, "cpp"):
                yield chunk  # Stream each chunk
        elif selected_language == "Java":
            for chunk in optimize(python_code, model, "java"):
                yield chunk
        return ""

    # Function to execute converted code
    def run_code(converted_code, selected_language):
        if selected_language == "C++":
            return execute_cpp(converted_code)
        elif selected_language == "Java":
            return execute_java(converted_code)
        return "Invalid language selection"

    sample_program.change(select_sample_program, inputs=[sample_program], outputs=[python])
    convert.click(convert_code, inputs=[python, model, language], outputs=[converted_code])
    converted_run.click(run_code, inputs=[converted_code, language], outputs=[output])   
    python_run.click(execute_python, inputs=[python], outputs=[python_out])

ui.launch(inbrowser=True)